In [65]:
from utils import *
from rich import print as rprint
import os
from numba import jit, njit
import copy
import utils
#import statscols

In [66]:
# load h5py data
data = load_ims(
    "/home/shehan/Documents/nih_projects/parsers/data/multi_surface_track_parser_dev_data/GFP #1 Sec2 Roi1 2x2 1h30min.ims"
)

In [67]:
for k, v in data.get('Scene8').items():
    print(k, v)

Content <HDF5 group "/Scene8/Content" (8 members)>
Data <HDF5 dataset "Data": shape (1,), type "|O">


In [68]:
for k, v in data.get('Scene8').get('Content').items():
    print(k, v)

MegaSurfaces0 <HDF5 group "/Scene8/Content/MegaSurfaces0" (40 members)>
MegaSurfaces1 <HDF5 group "/Scene8/Content/MegaSurfaces1" (40 members)>
MegaSurfaces2 <HDF5 group "/Scene8/Content/MegaSurfaces2" (40 members)>
MegaSurfaces3 <HDF5 group "/Scene8/Content/MegaSurfaces3" (40 members)>
MegaSurfaces4 <HDF5 group "/Scene8/Content/MegaSurfaces4" (40 members)>
MegaSurfaces5 <HDF5 group "/Scene8/Content/MegaSurfaces5" (40 members)>
MegaSurfaces6 <HDF5 group "/Scene8/Content/MegaSurfaces6" (40 members)>
MegaSurfaces7 <HDF5 group "/Scene8/Content/MegaSurfaces7" (40 members)>


In [69]:
for k, v in data.get('Scene8').get('Content').get("MegaSurfaces1").items():
    print(k, v)

BlockData <HDF5 dataset "BlockData": shape (280, 131072), type "<i8">
BlockInfo <HDF5 dataset "BlockInfo": shape (94816,), type "|V72">
BlockPath <HDF5 dataset "BlockPath": shape (0,), type "|V8">
Category <HDF5 dataset "Category": shape (3,), type "|V520">
CategoryFunction <HDF5 dataset "CategoryFunction": shape (1,), type "|V520">
CreationParameters <HDF5 dataset "CreationParameters": shape (1,), type "|S3506">
Factor <HDF5 dataset "Factor": shape (69,), type "|V520">
FactorFunction <HDF5 dataset "FactorFunction": shape (198,), type "|V520">
FactorList <HDF5 dataset "FactorList": shape (44,), type "|V8">
FactorListFunction <HDF5 dataset "FactorListFunction": shape (42,), type "|V8">
LabelColor <HDF5 dataset "LabelColor": shape (1,), type "|V12">
LabelColorData <HDF5 dataset "LabelColorData": shape (2,), type "|V272">
LabelColorLabelGroupNames <HDF5 dataset "LabelColorLabelGroupNames": shape (2,), type "|V264">
LabelColorLabelValues <HDF5 dataset "LabelColorLabelValues": shape (2,), t

In [79]:
np.array(data.get('Scene8').get('Content').get("MegaSurfaces1").get('LabelColorData'))

array([(b'Surface', 0, 0), (b'Track', 1, 0)],
      dtype=[('LabelCategoryName', 'S256'), ('LabelID', '<i8'), ('ColorID', '<i8')])

In [81]:
np.asarray(data.get('Scene8').get('Content').get("MegaSurfaces1").get("LabelColorLabelValues"))

array([(b'Surface_UnlabelName',), (b'Track_UnlabelName',)],
      dtype=[('LabelValue', 'S256')])

In [73]:
# np.asarray(data.get('Scene8').get('Content').get("MegaSurfaces0").get("FactorFunction"))

In [ ]:
stats_names = np.asarray(data["Scene8"]["Content"]["MegaSurfaces0"]["StatisticsType"])
stats_names

In [ ]:
stats_values = pd.DataFrame(
    np.asarray(data["Scene8"]["Content"]["MegaSurfaces0"]["StatisticsValue"])
)
stats_values

In [ ]:
track_ids = pd.DataFrame(
    np.asarray(data["Scene8"]["Content"]["MegaSurfaces0"]["Track0"])
)["ID"]
type(track_ids)

In [ ]:
# get stats dict for whole ims file
full_stats_dict = track_utils.get_statistics_dict(data, "MegaSurfaces0")
invert_stats_dict = track_utils.invert_stats_dict(full_stats_dict)

In [ ]:
# filter dataset to only have tracks
track_only_df = stats_values[stats_values["ID_Object"].isin(track_ids)]

In [ ]:
# filter the full stats dict to only have the info from the tracks
filtered_stats = {}
for stat_type in track_only_df["ID_StatisticsType"].to_list():
    filtered_stats[stat_type] = full_stats_dict[stat_type]

full_stats_dict

In [ ]:
inv_filtered = track_utils.invert_stats_dict(filtered_stats)

In [ ]:
invert_stats_dict['Time Index']

In [ ]:
full_stats_dict = track_utils.get_statistics_dict(data, "MegaSurfaces0")
full_stats_dict[16873]

In [ ]:
f = stats_values[stats_values["ID_StatisticsType"] == 16873]
first_objects = f[f["Value"] == 1]['ID_Object']

In [ ]:
stats_values[stats_values['ID_Object'].isin(first_objects)]

In [ ]:
track_and_object_id_info = track_utils.convert_to_matrix(
    track_id_data, track_object_data
)

In [ ]:
temp_dict = track_utils.create_dict(track_and_object_id_info)

In [ ]:
stats_data = track_utils.extract_data(track_and_object_id_info, stats_values)

In [ ]:
stats_dict[16969]

In [ ]:
inv_stats_dicts = track_utils.invert_stats_dict(stats_dict)

In [ ]:
inv_stats_dicts["Time"]

In [ ]:
statscols = statscols.StatsColumns(
    0,
    stats_dict,
    track_id_data,
    track_object_data,
    stats_values,
    track_and_object_id_info,
    stats_data,
    inv_stats_dicts,
)

In [ ]:
statscols

In [ ]:
## can we get surfaces by modifying following function
def get_object_names(full_data_file: h5py.File, search_for: str) -> list:
    """
    _summary_

    Args:
        full_data_file (h5py._hl.files.File): full imaris file in h5py File format
        search_for (str): string containing full or partial filename to search for

    Returns:
        (list): a list of all the object names that match search_for parameter
    """

    values = full_data_file.get("Scene").get("Content").keys()
    storage = list()
    for item in values:
        if len(re.findall(search_for, item)):
            storage.append(item)
    return storage


# test function -- working
out = get_object_names(data, "Surface")
print(out)

In [ ]:
# once we know the object names we want to extract data from use following function
# this function doesnt grab any data values, just names
def get_statistics_names(full_data_file: h5py.File, object_name: str) -> dict:
    """
    for a given object_name, extracts the statistics names and ids into a dict
    ex: statistics name = mean intensity, associated id=404

    Args:
        full_data_file (h5py._hl.files.File): full imaris file in h5py File format
        object_name (str): name of the object to get statistic names from

    Returns:
        dict: a dict where the keys=unique stats ID, value=static name
    """

    # get object specific data
    obj_specific_data = full_data_file["Scene8"]["Content"][object_name]

    # rearrange data
    statistics_name = np.asarray(obj_specific_data["StatisticsType"])
    statistics_name = pd.DataFrame(statistics_name)

    # extract statistics names
    stats_name = statistics_name["Name"]

    # extract statistics ID names
    stats_type = statistics_name["ID"]

    # combine stats type and stats names
    return dict(zip(stats_type, stats_name))


# test -- working
out_stats_names = get_statistics_names(data, "MegaSurfaces0")

out_stats_names

In [ ]:
# np.savetxt('test.txt', list(out_stats_names.values()), fmt='%s')

In [ ]:
# once we have the statistics names we can get numerical statistics values
# for each object id within the specified object
def get_stats_values(full_data_file: h5py.File, object_name: str) -> pd.DataFrame:
    """
    for a given object_name, extracts the statistics values for all object ids
    within the object

    Args:
        full_data_file (h5py._hl.files.File): full imaris file in h5py File format
        object_name (str): name of the object to get statistic names from

    Returns:
        pd.DataFrame: a pandas data frame that contains information about each object id
        where each object id has a stats id and associated stats value.
    """
    obj_specific_stats = full_data_file.get("Scene8").get("Content")[object_name][
        "StatisticsValue"
    ]
    obj_specific_stats = np.asarray(obj_specific_stats)
    return pd.DataFrame(obj_specific_stats)


out = get_stats_values(data, "MegaSurfaces0").drop("ID_Time", axis=1)
print(out)

In [ ]:
out[out["ID_StatisticsType"] == 16991].sort_values("ID_Object")

In [ ]:
extracted_stats = (
    out.groupby("ID_Object")[["ID_StatisticsType", "Value"]]
    .apply(lambda x: x.set_index("ID_StatisticsType").to_dict(orient="dict"))
    .to_dict()
)
extracted_stats = {
    k: v["Value"] for k, v in extracted_stats.items()
}  # clean up step for line above

In [ ]:
def invert_stats_dict(stats_dict: dict = None):
    """
    creates a inverted_stats_dict --> [Statistics Type: Numeric Value]
    """

    # sort the dict
    sorted_dict_key = sorted(stats_dict.keys())
    stats_dict = {key: str(stats_dict[key]).strip("b")[1:-1] for key in sorted_dict_key}

    # create an empty dictionary
    storage = {}

    for key in stats_dict.keys():
        # if the word is not in the new storage dict
        if stats_dict[key] not in storage.keys():
            storage[stats_dict[key]] = key
        else:
            # get the value inside the key
            current_value = storage[stats_dict[key]]

            # if its a single value create a dict else create a dict
            if type(current_value) != dict:
                # then its the first value
                storage[stats_dict[key]] = {}
                storage[stats_dict[key]]["channel_1"] = current_value

                # current value
                storage[stats_dict[key]]["channel_2"] = key
            else:
                # get the length of the dict
                count = len(current_value.keys())
                # updated count
                count += 1
                # create new key
                new_key = f"channel_{count}"
                current_value[new_key] = key
                storage[stats_dict[key]] = current_value

    return storage


# we can directly use this to invert the dict
# this inverted dict can be used to create the final excel file

In [ ]:
inverted_dict = invert_stats_dict(out_stats_names)
categories = [
    "Position X",
    "Position Y",
    "Position Z",
    "Intensity Mean_channel_1",
    "Intensity Mean_channel_2",
    "Intensity Mean_channel_3",
    "Intensity Mean_channel_4",
    "Intensity Mean_channel_5",
    "Volume",
]

In [ ]:
# flatten dict
from collections.abc import MutableMapping


def flatten(d, parent_key="", sep="_"):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, MutableMapping):
            items.extend(flatten(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

In [ ]:
inverted_flatten = flatten(inverted_dict)
# can we provide an editable list before anything starts so he can manually select which ones he wants
# that way the names match exactly because I control it.

# reverse dict again, key=num, value=name
final_dict = {v: k for k, v in inverted_flatten.items()}

# can we use the categories reqested to create a list of numbers we dont want?
del_list = [None] * (len(inverted_flatten) - len(categories))
counter = 0
for idx, (k, v) in enumerate(inverted_flatten.items()):
    if k not in categories and v != None:
        del_list[counter] = v
        counter += 1

In [ ]:
final_dict[170]

In [ ]:
# map what the user wants to numeric values
user_request_numeric = [inverted_flatten[x] for x in categories]

# we can look at the final dict where key=num, value=name to filter
final_dict_copy = copy.deepcopy(final_dict)
for value in user_request_numeric:
    final_dict_copy.pop(value)

new_del_list = list(final_dict_copy.keys())

In [ ]:
print(len(final_dict_copy.keys()), len(final_dict.keys()), len(user_request_numeric))

In [ ]:
final_df = pd.DataFrame(extracted_stats).transpose()
f = final_df.drop(labels=del_list, axis=1)
print(f)

In [ ]:
final_df = pd.DataFrame(extracted_stats).transpose()
f = final_df.drop(labels=[None], axis=1)
print(f)

In [ ]:
# clean up df by changing number to names and rearranging columns
columns = f.columns
print(columns)

new_names = {key: final_dict[key] for key in columns}
print(new_names)

# rename dict

In [ ]:
final_df = f.rename(new_names, axis=1)
print(final_df.head())

In [ ]:
# create the final id column
final_df["ID"] = final_df.index


# rearrange
final_order = [
    "ID",
    "Position X",
    "Position Y",
    "Position Z",
    "Intensity Mean_channel_1",
    "Intensity Mean_channel_2",
    "Intensity Mean_channel_3",
    "Intensity Mean_channel_4",
    "Intensity Mean_channel_5",
]

final = final_df[final_order]
print(final)